# Recurrent Neural Networks

## 1 - Introduction

The objective of this lab session is to design and evaluate systems for character-by-character text prediction. These models are capable, given a few input characters, to generate text of arbitrary length.

To do so, the study will focus on deep learning models for sequence processing known as recurrent neural networks. We have a dataset representing part of Shakespeare's works, in the form of a plain text file containing about 1.1 million characters (~ 1 Mo). The role of the learning process is thus to correctly predict each character given a sequence of past information.

To model the relationship between characters in a text, we will first focus on the construction of a simple recurrent neural network along with its training and testing procedures. A second part will compare the performance of the RNN to that of its variants, the Long Short-Term Memory (LSTM) and Gated Recurrent Units (GRU) on this text synthesis task.

This lab session uses the Python 3 version of the Tensorflow framework, for which an API is available at the adress:
https://www.tensorflow.org/api_docs/python/


## 2 - RNN construction

Sequences are series of elements that, unless random, are structured and often contain regularities. As a result, elements of a sequence can be modeled in relation to others, with invariance properties that the commonly used one-to-one architectures fail to capture.

A solution to this problem is the addition of recurrence to this type of models to obtain recurrent neural networks. The RNN introduces a hidden recurrent state that, at each timestep, contains deterministic information from the past sequence elements. Its transformation parameters are shared along time, so that each part of the input sequence is treated identically. Compared to other parameter-sharing models such as CNNs, the RNN has theoretically unlimited memory and can process sequences of varying lengths. A recurrent graph can be unrolled in time as equivalent to a multilayer perceptron, and gradient backpropagation is thus performed using the same method (chain rule).

In practice, the implementation of the RNN consists of three different modules similar to other deep learning models:
 - The definition of the model, including its architecture and parameters, its transformations and its optimization details
 - The learning process for batches of paired input-output examples
 - A testing function used for generation, after training
  
In this part we will implement each of the steps needed to train and test a simple RNN model for character generation. To reduce computational costs only 10% of the available dataset will be used for training, the impact of which will be discussed in part 3.

### 2.1 - Model

In Tensorflow, the architecture and the optimization process can be defined in the model in the form of variable containers and operations. During training, they are evaluated as a function of the input and target output. The model definition thus includes several essential parts:
 - The pre-processing and formatting of inputs and outputs
 - The initialization of model parameters (weights)
 - The forward operations apllied to inputs using the previously defined parameters
 - The definition and computation of a loss function
 - An optimization procedure to apply parameter updates as a function of backpropagated gradients


The skeleton of the code as well as some of the operations are already implemented below. Complete the different parts where indicated using the suggested functions in parenthesis as well as the Tensorflow API:
 - Placeholders for input and output (tf.Placeholder) are containers which will receive data examples during training and testing.
 - Input embeddings (tf.get_variable, tf.nn.embedding_lookup) which transform each input character into a vector. In the case of categorical (discrete) input data, embeddings are often preferred as an alternative to one-hot encoding. The embedding process maps each character to a learned real-valued vector of arbitrary dimension. Here we choose rnn_size as the embedding dimension for simplicity.
 - The definition of the RNN cells (rnn.BasicRNNCell, rnn.MultiRNNCell) with size parameters to initialize weights
 - The RNN forward process (legacy_seq2seq.rnn_decoder) implements the recurrent transformations given an input sequence
 - Output softmax probabilities (tf.matmul, tf.nn.softmax) can be computed from the RNN last layer's internal state. A linear transform first maps the RNN output to a real-valued vector, then input to the softmax to obtain a probability value for each character in the vocabulary.
 - A loss function (legacy_seq2seq.rnn_loss_by_example) compares each output character with its ground truth, the training process aims at maximizing the probability of choosing the right character while minimizing others.
 - Loss reduction along time and examples (tf.reduce_sum) is a way to compute a single cost value by averaging the error from all outputs.

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq

import numpy as np

class Model():
    def __init__(self, batch_size, seq_length, vocab_size, training=True):
        rnn_size = 128
        num_layers = 1
        
        if not training:
            batch_size = 1
            seq_length = 1
        
        # ----- Inputs-Outputs -----
        # Placeholders for input and output data - To complete
        self.input_data = ...
        self.targets = ...
        
        # Input embeddings - To complete
        embedding = ...
        inputs = ...

        inputs = tf.split(inputs, seq_length, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
        
        
        # ----- Model parameters definition -----
        # Definition of the recurrent cells - To complete
        cells = []
        for _ in range(num_layers):
            cells.append(...)
        self.cell = cell = ...
        
        # RNN state to output transformation
        with tf.variable_scope('rnnlm'):
            softmax_w = tf.get_variable("softmax_w",
                                        [rnn_size, vocab_size])
            softmax_b = tf.get_variable("softmax_b", [vocab_size])

        
        # ----- Model forward definition -----
        # Initial state
        self.initial_state = cell.zero_state(batch_size, tf.float32)
        
        # Loop used during generation (test) only
        def loop(prev, _):
            prev = tf.matmul(prev, softmax_w) + softmax_b
            prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
            return tf.nn.embedding_lookup(embedding, prev_symbol)

        # RNN implementation - To complete
        outputs, last_state = ...
        
        self.final_state = last_state
        output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])

        # Output softmax probabilities - To complete
        self.logits = ...
        self.probs = ...
        
        # ----- Loss computation -----
        # Loss function - To complete
        loss = ...
        # Loss reduction - To complete
        with tf.name_scope('cost'):
            self.cost = ...
        
        # ---- Optimization -----
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), 5.)
        with tf.name_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def sample(self, sess, chars, vocab, num=200, prime='The '):
        # Initialize the internal state at the first timestep
        state = sess.run(self.cell.zero_state(1, tf.float32))
        # Process the prime sequence
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [state] = sess.run([self.final_state], feed)

        # Sampling in a softmax distribution
        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        # Character-by-Character generation
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]
            sample = weighted_pick(p)
            pred = chars[sample]
            ret += pred
            char = pred
        return ret

### 2.2 - Training

The role of the training process is to optimize the previously implemented model on a given dataset. As such, it operates on a specific instance of the model's class with set dimensions. To facilitate the segmentation and rotation of training data a TextLoader class is implemented in the Python script "utils.py".

Implement where indicated the instantiation of the TextLoader and Model classes using the proposed meta-parameters.

In [ ]:
import os
from six.moves import cPickle
from utils import TextLoader

tf.reset_default_graph()
tf.set_random_seed(100)

data_dir = 'data/tinyshakespeare_10'
save_dir = 'save'
log_dir = 'logs'
batch_size = 32
seq_length = 50
num_epochs = 10

# Initialize the data loader - To complete
data_loader = ...
vocab_size = data_loader.vocab_size

# Initialize the RNN model - To complete
model = ...

# Save dictionary and parameters
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'config.pkl'), 'wb') as f:
    cPickle.dump((batch_size, seq_length, vocab_size), f)
with open(os.path.join(save_dir, 'chars_vocab.pkl'), 'wb') as f:
    cPickle.dump((data_loader.chars, data_loader.vocab), f)

Once the model is initialized, the training process can be implemented. In practice, training is performed inside a Tensorflow session that can be run to evaluate the outputs and optimization-related quantities given an input sequence and its ground truth output. At each epoch (iteration over the whole dataset), each batch of input-output examples are passed through the network and backpropagation is used to apply small changes to the model's parameters.

As the different processing steps are implemented in the model, the training implementation only needs to cover data management and run the model. Complete the code below using the suggested functions:

 - Set the model's learning rate to 0.002 (sess.run, tf.assign)
 - Reset the batch pointer at each new epoch, corresponding to an iteration over the whole dataset (see utils.py)
 - Load the current batch of data examples (see utils.py and Tensorflow placeholders)
 - Run the model to perform an optimization step on the current batch, and return the loss (sess.run)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    # Set the model learning rate to 0.002 - To complete
    sess.run(tf.assign(model.lr, 0.002))
    
    losses = []
    for e in range(num_epochs):
        # Reset the batch pointer - To complete
        ...
        
        # Compute the initial state
        state = sess.run(model.initial_state)
        for b in range(data_loader.num_batches):
            # Load the current batch of data - To complete
            ...
            
            # Run the model for the current batch - To complete
            ...
            
            losses.append(train_loss)
            print("{}/{} (epoch {}), train_loss = {:.3f}"
                  .format(e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          e, train_loss))
    # Save the trained model
    checkpoint_path = os.path.join(save_dir, 'model.ckpt')
    saver.save(sess, checkpoint_path,
               global_step=e * data_loader.num_batches + b)
    print("model saved to {}".format(checkpoint_path))

The loss at each batch is saved in the losses list. Display the corresponding loss curve using Matplotlib and comment on its evolution.

In [ ]:
...

### 2.3 - Sampling

Now that the model is trained and saved, we can use it to generate text sequences of arbitrary length. This is implemented in the sample() function, which takes two arguments:
 - The length of the sequence to generate
 - The prime sequence that we want to build a text upon

This function must first construct the model and load the learnt parameters before generating and saving the desired text sequence. Complete the definition of the function with:
 - The model initialization with training mode disabled
 - The generation procedure for the defined parameters
 

In [ ]:
import codecs

def sample(n = 1000, prime = u' '):
    tf.reset_default_graph()
    save_dir = 'save'
    sample = 1

    tf.set_random_seed(100)
    
    # Load parameters and dictionary
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        batch_size, seq_length, vocab_size = cPickle.load(f)
    with open(os.path.join(save_dir, 'chars_vocab.pkl'), 'rb') as f:
        chars, vocab = cPickle.load(f)
        
    # Initialize the model - To complete
    ...
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        # Load the model
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            
            # Sample - To complete
            ...
            
            # Write the generated text to a file and to the console
            with codecs.open(os.path.join(save_dir, "sample.txt"), "w", "utf-8-sig") as temp:
                temp.write(sample_test)

            print(sample_test)

Test the sample() function using a large sequence length (eg: 1000). Describe qualitatively the performance of the network.

In [ ]:
...

## 3 - Comparison of recurrent models

Simple recurrent neural networks suffer from problems related to the transmission of long-term information. During backpropagation, the gradient is multiplied by the recurrent weight matrix for each timestep. Depending on the spectral radius (i.e. largest singular value) of this matrix, the gradient can exponentially explode or vanish making the learning of long-term dependancies difficult. As these parameters are learned, there is no way to ensure that gradient vanishing will not impair the performance of the network.
Variants of the basic RNN cell have been proposed that ensure a good flow of information along time via the "constant error flow" principle: the long short-term memory (LSTM) and gated recurrent units (GRU). These models tend to better capture long-term structures in sequences at the cost of increased computational and memory complexities.

Additionally, the performance of deep learning models heavily depends on the dimensions of the architectures, which is chosen semi-arbitrarily, as well as on both the quantity and quality of the available data.

### 3.1 - Qualitative evaluation of generated sample quality

Several models have been trained to compare the quality of generated text based on a set of factors:
 - Recurrent cell types (RNN, LSTM, GRU)
 - Dataset size (10p - 100k characters, 1M characters)
 - Internal state dimensions (128, 512)
 - Number of recurrent layers (1, 3)

The corresponding models are saved in folders using the following name structure: 

save_[RNN type]_ [# layers]_ [Internal state size]_ [Training epochs]_ [Dataset size]

Samples can be obtained using the sample() function defined in sample.py (example below).

Generate long text sequences for each trained model and qualitatively compare the results as a function of the experimental factors described above. Examples are saved in a "sample.txt" file in the model folder.

In [ ]:
from sampling.sample import sample as sample2
sample2('save_rnn_1_128_10e_10p', 2000)
...

### 3.2 - Quantitative evaluation of generated sample quality

The only objective quantitative metric available to evaluate the performance of generative models is the cost function, which the learning process tries to minimize. However, it is often insufficient when modeling complex structures such as written language, for which several implicit rules must be observed (grammar, sentence structure...).

 - Propose a method to evaluate the quality of generated extracts, in the form of a quantitative indicator (0-1 scale). Implement it and apply it to generated sequences from the trained models. For this part you can use an external Python script or Jupyter Notebook.
 

In [ ]:
# Quantitative indicator implementation
...